In [54]:
import textwrap
from IPython.display import Markdown, display
from langchain_text_splitters import SentenceTransformersTokenTextSplitter
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import chromadb
import os
from chromadb.utils import embedding_functions
from chromadb import Client
from tqdm.notebook import trange, tqdm
from pprint import pprint



In [55]:
def convert_PDF_Text(pdf_path="/Users/harun/Documents/GitHub/AI-Powered-Regional-Cost-of-Living-Advisor/Search/Rag/Uni_fiyatları/istanbulGalataUniversitesi.pdf"):
    reader = PdfReader(pdf_path)
    pdf_texts = [p.extract_text().strip() for p in reader.pages]
    # Filter the empty strings
    pdf_texts = [text for text in pdf_texts if text]
    return pdf_texts

pdf_texts=convert_PDF_Text()
pdf_texts

['İstanbul Galata Üniversitesi Eğitim Ücretleri ve Bursları 2024-2025\nDiş Hekimliği Fakültesi Ücretleri\nDiş Hekimliği (Ücretli): 780.000 TL (%50 İndirimli): 390.000 TL\nDiş Hekimliği İngilizce (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL\nSanat ve Sosyal Bilimler Fakültesi Ücretleri\nYönetim Bilişim Sistemleri (egitim.net.tr) (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL\nPsikoloji (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL\nPsikoloji İngilizce (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL\nGastronomi ve Mutfak Sanatları (Ücretli): 550.000 TL (%50 İndirimli): 275.000 TL\nİletişim ve Tasarımı (%50 İndirimli): 225.000 TL\nİşletme (Ücretli): 450.000 TL (%50 İndirimli): 225.000 TL\nHalkla İlişkiler ve Reklamcılık (%50 İndirimli): 187.500 TL\nİç Mimarlık ve Çevre Tasarımı (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL\nİç Mimarlık ve Çevre Tasarımı İngilizce (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL\nMühendislik Fakültesi Ücretleri\nBilgisayar Mühendisliği (

In [56]:
# Helper for displaying text
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))



In [57]:
def text_Chunks_in_Char(pdf_texts, chunk_size=500, chunk_overlap=50):
    # Combine all PDF texts into one string
    full_text = '\n\n'.join(pdf_texts)

    # Initialize sentence-based text splitter
    sentence_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", ":", ";", ",", " ", ""],  # Process in this order
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    # Split text into chunks based on sentences
    chunks = sentence_splitter.split_text(full_text)
    return chunks
chunks=text_Chunks_in_Char(pdf_texts)
print(len(chunks))
chunks

10


['İstanbul Galata Üniversitesi Eğitim Ücretleri ve Bursları 2024-2025\nDiş Hekimliği Fakültesi Ücretleri\nDiş Hekimliği (Ücretli): 780.000 TL (%50 İndirimli): 390.000 TL\nDiş Hekimliği İngilizce (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL\nSanat ve Sosyal Bilimler Fakültesi Ücretleri\nYönetim Bilişim Sistemleri (egitim.net.tr) (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL\nPsikoloji (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL',
 'Psikoloji İngilizce (Ücretli): 660.000 TL (%50 İndirimli): 330.000 TL\nGastronomi ve Mutfak Sanatları (Ücretli): 550.000 TL (%50 İndirimli): 275.000 TL\nİletişim ve Tasarımı (%50 İndirimli): 225.000 TL\nİşletme (Ücretli): 450.000 TL (%50 İndirimli): 225.000 TL\nHalkla İlişkiler ve Reklamcılık (%50 İndirimli): 187.500 TL\nİç Mimarlık ve Çevre Tasarımı (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL\nİç Mimarlık ve Çevre Tasarımı İngilizce (Ücretli): 570.000 TL (%50 İndirimli): 285.000 TL',
 'Mühendislik Fakültesi Ücretleri\nBilgisayar Mühendis

In [58]:
sentence_transformer_model = "distiluse-base-multilingual-cased-v1"

In [59]:

def convert_Chunk_Token(text_chunksinChar, sentence_transformer_model, chunk_overlap=10, tokens_per_chunk=128):
    token_splitter = SentenceTransformersTokenTextSplitter(
        chunk_overlap=chunk_overlap,
        model_name=sentence_transformer_model,
        tokens_per_chunk=tokens_per_chunk)

    text_chunksinTokens = []
    for text in text_chunksinChar:
        text_chunksinTokens += token_splitter.split_text(text)
    print(f"\nTotal number of chunks (document splited by 128 tokens per chunk): {len(text_chunksinTokens)}")
    return text_chunksinTokens
tokens=convert_Chunk_Token(chunks,sentence_transformer_model)
tokens[7]


Total number of chunks (document splited by 128 tokens per chunk): 19


'TL ( % 50 İndirimli ) : 195. 000 TL'

In [60]:

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)


In [61]:
def create_chroma_client(collection_name, embedding_function):
    chroma_client = chromadb.PersistentClient()

    # Check if collection already exists and delete it
    existing_collections = [col.name for col in chroma_client.list_collections()]
    if collection_name in existing_collections:
        chroma_client.delete_collection(collection_name)

    # Create a new collection
    chroma_collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=embedding_function
    )

    return chroma_client, chroma_collection



In [62]:
def add_meta_data(chunks, title, category, initial_id):
    ids = [str(i + initial_id) for i in range(len(chunks))]
    filename = os.path.basename(title)
    metadata = {
        'document': filename,
        'category': category
    }
    metadatas = [metadata for i in range(len(chunks))]
    return ids, metadatas


In [63]:
def add_document_to_collection(ids, metadatas, chunks, chroma_collection):
    chroma_collection.upsert(ids=ids, metadatas=metadatas, documents=chunks)
    return chroma_collection

In [64]:
def retrieveDocs(chroma_collection, query, file=None, n_results=10, return_only_docs=False):

    # Build query parameters
    query_params = {
        "query_texts": [query],
        "include": ["documents", "metadatas", 'distances'],
        "n_results": n_results
    }

    # Add file filter if provided
    if file is not None:
        query_params["where"] = {"document": f"{file}.pdf"}

    # Execute the query
    results = chroma_collection.query(**query_params)
    print(results['distances'])
    if return_only_docs:
        return results['documents'][0]
    else:
        return results


In [65]:
def show_results(results, return_only_docs=False):
    if return_only_docs:
        retrieved_documents = results

        if len(retrieved_documents) == 0:
            return []
        # Format each document with a bullet point
        formatted_docs = [f"• {doc}" for doc in retrieved_documents]
        return "\n".join(formatted_docs)
    else:
        retrieved_documents = results['documents'][0]
        metadatas = results['metadatas'][0]

        if len(retrieved_documents) == 0:
            return []

        # Format each document with source information
        formatted_docs = []
        for doc, metadata in zip(retrieved_documents, metadatas):
            # Clean up the document text by removing extra spaces and [UNK] tokens
            cleaned_doc = doc.replace('[UNK]', '').strip()
            # Add proper spacing between sentences
            cleaned_doc = '. '.join(s.strip() for s in cleaned_doc.split('.') if s.strip())
            formatted_doc = f"• {cleaned_doc}\n  From: {metadata['document']}\n"
            formatted_docs.append(formatted_doc)
        return "\n".join(formatted_docs)


In [66]:

def list_files_in_directory(directory_path):
    all_entries = os.listdir(directory_path)

    files_only = [entry for entry in all_entries
                  if os.path.isfile(os.path.join(directory_path, entry))]
    return files_only


In [67]:
def get_all_pdf_paths(
        directory_path="/Users/harun/Documents/GitHub/AI-Powered-Regional-Cost-of-Living-Advisor/Search/Rag/Uni_fiyatları"):
    """
    Get all PDF file paths from the specified directory.

    Args:
        directory_path (str): Path to the directory containing PDF files

    Returns:
        list: List of full paths to all PDF files in the directory
    """
    all_files = list_files_in_directory(directory_path)
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]
    full_paths = [os.path.join(directory_path, pdf_file) for pdf_file in pdf_files]
    return full_paths


In [68]:
def load_multiple_pdfs_to_ChromaDB(collection_name, sentence_transformer_model):
    """
    Load multiple PDFs into ChromaDB collection with proper chunking and embedding.

    Args:
        collection_name (str): Name of the ChromaDB collection
        sentence_transformer_model (str): Name of the sentence transformer model to use

    Returns:
        tuple: (chroma_client, chroma_collection)
    """
    # Initialize embedding function
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)

    # Create or get ChromaDB client and collection
    chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function)

    # Get current ID for continuous indexing
    current_id = chroma_collection.count()

    # Get all PDF paths from the directory
    pdf_paths = get_all_pdf_paths()

    for pdf_path in pdf_paths:
        # Extract text from PDF
        pdf_texts = convert_PDF_Text(pdf_path)

        # Create character-based chunks first
        char_chunks = text_Chunks_in_Char(pdf_texts)
        
        # Convert to token-based chunks
        token_chunks = convert_Chunk_Token(char_chunks, sentence_transformer_model)

        # Add metadata and get IDs
        ids, metadatas = add_meta_data(token_chunks, pdf_path, "PricePaper", current_id)

        # Update current_id for next document
        current_id += len(token_chunks)

        # Add to collection
        chroma_collection = add_document_to_collection(ids, metadatas, token_chunks, chroma_collection)

    return chroma_client, chroma_collection

In [69]:
def get_existing_chroma_collection(collection_name):

    chroma_client = chromadb.PersistentClient()
    
    # Get the existing collection
    chroma_collection = chroma_client.get_collection(
        name=collection_name
    )
    
    return  chroma_collection

In [72]:
from KeywordAgent import keyword_agent

# Test code
# chroma_client, chroma_collection = load_multiple_pdfs_to_ChromaDB("UniPrices", sentence_transformer_model)
chroma_collection=get_existing_chroma_collection("UniPrices")


chroma_collection = get_existing_chroma_collection("UniPrices")
query = "Diyetetik Ücretleri"
keyword = "istanbulGalataUniversitesi"
filtered_keyword = keyword.strip('"')
retrieved_documents = retrieveDocs(chroma_collection, query, filtered_keyword, 10, return_only_docs=True)
print(show_results(retrieved_documents,return_only_docs=True))
# keyword = keyword_agent.parse_keywords(query)
# filtered_keyword = keyword.strip('"')
# if filtered_keyword != 'None':
#     retrieved_documents = retrieveDocs(chroma_collection, query, "istanbulGalataUniversitesi", 10)
#     show_results(retrieved_documents)
# else:
#     retrieved_documents = retrieveDocs(chroma_collection, query, None, 10)
#     show_results(retrieved_documents)



InternalError: Database error: error returned from database: (code: 1) no such table: tenants